# **STEP 12:** Organize Isodose CSV Dataset

In [15]:
#the python debugger is a library that provides an interactive source code debugger for Python programs
import pdb

import radiomics
#from radiomics import featureextractor
import os
import csv
import pandas as pd

In [12]:
from pathlib import Path
import re, csv

root_dir = Path("/notebooks/disk2/RT_Dataset_2/06_Processed_Patients")
out_csv  = Path("/notebooks/disk2/CSV_Dataset/01_Isodose.csv")

# ----- helpers -----
exclude_patients = {"Patient_55", "Patient_56", "Patient_57", "Patient_70"}
roi_pat = re.compile(r"^mask_(.+?)\.nii\.gz$", re.IGNORECASE)

def pnum(patient_dir_name: str) -> str:
    m = re.search(r"Patient_(\d+)", patient_dir_name)
    return m.group(1) if m else ""

def ct_scan_name(pn: str, patient_dir_name: str) -> str:
    # special-case: 55–57 are Windowed only
    if patient_dir_name in {"Patient_55", "Patient_56", "Patient_57"}:
        return f"P{pn}_CT_1_TRATAMENTO_Windowed.nii.gz"
    return f"P{pn}_CT_1_TRATAMENTO_Windowed_Denoised.nii.gz"

def pick_best(files, preferred_suffix: str):
    """
    files: list[Path]
    preferred_suffix: '_treat.nii.gz' / '_fu.nii.gz' / '_ct.nii.gz'
    Returns Path | None
    """
    if not files:
        return None
    # exact preferred suffix
    for f in files:
        if f.name.endswith(preferred_suffix):
            return f
    # otherwise: if there is a plain one without _treat/_fu/_ct, prefer it
    for f in files:
        if not re.search(r'_(treat|fu|ct)\.nii\.gz$', f.name, flags=re.IGNORECASE):
            return f
    # else fallback to first (deterministic by sort earlier)
    return files[0]

def find_isodose(dir_path: Path, pn: str, roi: str, preferred_suffix: str) -> str | None:
    """
    Look inside dir_path for files like:
       P{pn}_Isodose_Volume_mask_{roi}_Resampled_Filtered*.nii.gz
    and pick the best according to preferred_suffix.
    Returns the filename (str) or None.
    """
    if not dir_path.exists():
        return None
    # try the strict pattern
    candidates = sorted(dir_path.glob(f"P{pn}_Isodose_Volume_mask_{roi}_Resampled_Filtered*.nii.gz"))
    if not candidates:
        # tolerate small ROI typos: collapse consecutive dashes/underscores and try again
        roi_relaxed = re.sub(r'[-_]+', '-', roi)
        candidates = sorted(dir_path.glob(f"P{pn}_Isodose_Volume_mask_{roi_relaxed}_Resampled_Filtered*.nii.gz"))
    best = pick_best(candidates, preferred_suffix)
    return best.name if best else None

# ----- CSV columns -----
COLUMNS = [
    "Patient_ID",
    "Mask_Name",
    "CT_Treatment_Path",
    "CT_Treatment_Volume",
    "MR_Treatment_Path",
    "MR_Treatment_Volume",
    "Follow_Up_Path",
    "Follow_Up_Volume",
    "Treatment_Isodose_Mask",   # from MR_Treatment_Path (prefer *_treat.nii.gz)
    "Follow_Up_Isodose_Mask",   # from Follow_Up_Path (prefer *_fu.nii.gz)
    "CT_Isodose_Mask",          # from CT_Treatment_Path (prefer *_ct.nii.gz)
]

# ----- build -----
out_csv.parent.mkdir(parents=True, exist_ok=True)
with out_csv.open("w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(COLUMNS)

    for patient_dir in sorted(root_dir.iterdir()):
        if not patient_dir.is_dir():
            continue
        name = patient_dir.name
        if name in exclude_patients or not name.startswith("Patient_") or name == ".ipynb_checkpoints":
            continue

        pn = pnum(name)
        if not pn:
            continue

        ct_dir = patient_dir / f"P{pn}_CT_1_TRATAMENTO"
        mr_dir = patient_dir / f"P{pn}_RM_1_TRATAMENTO"
        fu_dir = patient_dir / f"P{pn}_RM_1_FOLLOW_UP"

        ct_vol = ct_scan_name(pn, name)
        mr_vol = f"P{pn}_RM_1_TRATAMENTO_Registered_1.nii.gz"
        fu_vol = f"P{pn}_RM_1_FOLLOW_UP_Registered_1.nii.gz"

        # base ROI masks in CT folder: mask_<ROI>.nii.gz (exclude *_fu/_treat/_ct)
        base_masks = sorted(
            f for f in ct_dir.glob("mask_*.nii.gz")
            if not (f.name.endswith("_fu.nii.gz") or f.name.endswith("_treat.nii.gz") or f.name.endswith("_ct.nii.gz"))
        )

        for mfile in base_masks:
            m = roi_pat.match(mfile.name)
            if not m:
                continue
            roi = m.group(1)  # 'PTV-3x8-1cm', etc.

            treat_iso = find_isodose(mr_dir, pn, roi, "_treat.nii.gz")
            fu_iso    = find_isodose(fu_dir, pn, roi, "_fu.nii.gz")
            ct_iso    = find_isodose(ct_dir, pn, roi, "_ct.nii.gz")

            # Only write the row if we at least have the treatment mask present in MR folder
            if not treat_iso:
                # uncomment to log: print(f"⚠️ {name} ROI {roi}: treatment isodose not found in {mr_dir}")
                continue

            row = [
                name,
                mfile.stem,     # keep mask_<ROI> as in your CSV
                str(ct_dir),
                ct_vol,
                str(mr_dir),
                mr_vol,
                str(fu_dir),
                fu_vol,
                treat_iso,      # ✅ actual filename from MR_Treatment_Path
                fu_iso or "",   # actual filename from Follow_Up_Path (or empty if missing)
                ct_iso or "",   # actual filename from CT_Treatment_Path (or empty if missing)
            ]
            writer.writerow(row)

print(f"CSV written to: {out_csv}")


CSV written to: /notebooks/disk2/CSV_Dataset/01_Isodose.csv


In [18]:
df = pd.read_csv('/notebooks/disk2/CSV_Dataset/01_Isodose.csv')

exclude_ids = {'Patient_55', 'Patient_56', 'Patient_57', 'Patient_70'}

if 'Patient_ID' in df.columns:
    df = df[~df['Patient_ID'].astype(str).str.strip().isin(exclude_ids)].reset_index(drop=True)

df.head() 

Patient_ID                         Mask_Name  \
0  Patient_54       mask_PTV-frontal-D-17Gy.nii   
1  Patient_54   mask_PTV-frontal-E-anterior.nii   
2  Patient_54  mask_PTV-frontal-E-post-18Gy.nii   
3  Patient_54      mask_PTV-occiptal-E-18GY.nii   
4  Patient_58                mask_PTV-5x5Gy.nii   

                                   CT_Treatment_Path  \
0  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   
1  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   
2  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   
3  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   
4  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   

                            CT_Treatment_Volume  \
0  P54_CT_1_TRATAMENTO_Windowed_Denoised.nii.gz   
1  P54_CT_1_TRATAMENTO_Windowed_Denoised.nii.gz   
2  P54_CT_1_TRATAMENTO_Windowed_Denoised.nii.gz   
3  P54_CT_1_TRATAMENTO_Windowed_Denoised.nii.gz   
4  P58_CT_1_TRATAMENTO_Windowed_Denoised.nii.gz   

                                   MR_Treatment_Path  \
0  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   
1  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   
2  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   
3  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   
4  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   

                       MR_Treatment_Volume  \
0  P54_RM_1_TRATAMENTO_Registered_1.nii.gz   
1  P54_RM_1_TRATAMENTO_Registered_1.nii.gz   
2  P54_RM_1_TRATAMENTO_Registered_1.nii.gz   
3  P54_RM_1_TRATAMENTO_Registered_1.nii.gz   
4  P58_RM_1_TRATAMENTO_Registered_1.nii.gz   

                                      Follow_Up_Path  \
0  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   
1  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   
2  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   
3  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   
4  /notebooks/disk2/RT_Dataset_2/06_Processed_Pat...   

                         Follow_Up_Volume  \
0  P54_RM_1_FOLLOW_UP_Registered_1.nii.gz   
1  P54_RM_1_FOLLOW_UP_Registered_1.nii.gz   
2  P54_RM_1_FOLLOW_UP_Registered_1.nii.gz   
3  P54_RM_1_FOLLOW_UP_Registered_1.nii.gz   
4  P58_RM_1_FOLLOW_UP_Registered_1.nii.gz   

                              Treatment_Isodose_Mask  \
0  P54_Isodose_Volume_mask_PTV-frontal-D-17Gy_Res...   
1  P54_Isodose_Volume_mask_PTV-frontal-E-anterior...   
2  P54_Isodose_Volume_mask_PTV-frontal-E-post-18G...   
3  P54_Isodose_Volume_mask_PTV-occiptal-E-18GY_Re...   
4  P58_Isodose_Volume_mask_PTV-5x5Gy_Resampled_Fi...   

                              Follow_Up_Isodose_Mask  \
0  P54_Isodose_Volume_mask_PTV-frontal-D-17Gy_Res...   
1  P54_Isodose_Volume_mask_PTV-frontal-E-anterior...   
2  P54_Isodose_Volume_mask_PTV-frontal-E-post-18G...   
3  P54_Isodose_Volume_mask_PTV-occiptal-E-18GY_Re...   
4  P58_Isodose_Volume_mask_PTV-5x5Gy_Resampled_Fi...   

                                     CT_Isodose_Mask  
0  P54_Isodose_Volume_mask_PTV-frontal-D-17Gy_Res...  
1  P54_Isodose_Volume_mask_PTV-frontal-E-anterior...  
2  P54_Isodose_Volume_mask_PTV-frontal-E-post-18G...  
3  P54_Isodose_Volume_mask_PTV-occiptal-E-18GY_Re...  
4  P58_Isodose_Volume_mask_PTV-5x5Gy_Resampled_Fi...